# Detecting motifs by aligning sequences to PSSMs
Name: Yanfang Guo

Department : MACS

Email: Yanfang.Guo@vub.be

- here I use [Clustal Omega](http://www.ebi.ac.uk/Tools/msa/clustalo/) to align the WW-sequence
- all step are followed by these formulas
![](https://ww4.sinaimg.cn/large/006tNbRwgy1fdo5b0ccmoj30o80b0aas.jpg)
- some variables in the program
    - fua -->$f_{u,a}$
    - mua -->$m_{u,a}$
    - pa  -- >$p_{a}$
    - alpa -->$\alpha$
    - beta -->$\beta$
    - Nseq --> $N_{seq}$

In [1]:
from read_fasta import *
from Matrix import *
from numpy import *

- read the file
- Methods
    - print_mat2() nicely print 2-dimension data set
    - fre(b,pos) b is the specific amino acid, and pos is the specific position

In [2]:
blosum = Matrix("BLOSUM62.txt")
seq_aligned = read_fasta("WW-aligned-136.fasta")
seq2 =seq_aligned[0]

amino = ['A', 'Q', 'L', 'S', 'R', 'E', 'K', 'T', 'N', 'G', 'M', 'W', 'D', 'H', 'F', 'Y', 'C', 'I', 'P', 'V']
# print(amino)




def print_mat2(alist):
    for i in alist:
        for j in i:
            print('{0:2f}'.format(j), end=" ")
        print()


def fre(b, pos):
    count = 0
    for i in range(len(seq_aligned)):
        if b == seq_aligned[i][pos]:
            count += 1
    return count / len(seq_aligned)



def print_fua(dict):
    for key in dict:
        print(key, end=':')
        for j in dict[key]:
            print('{0:4f}'.format(j), end=" ")
        print()
    print()
    print()

## Calculate $f_{u,a}$


In [3]:
fua = {}
# calculate the fua
for i in amino:
    t = []
    for j in range(len(seq_aligned[0])):
        t.append(fre(i, j))
    fua[i] = t
# print_fua(fua)


#print_fua(fua)

## Calculate $p_{u,a}$ and $m_{u,a}$ according to the formula
- print out $p_{u,a}$ dictionary and compare that with weblogo constructed from the website


In [4]:
# construct the pa dictionary

f = open("pa.txt")
pa = {}
for line in f:
    t = line.split()
    for i in range(0, len(t), 2):
        tt = t[i]
        pa[tt] = float(t[i + 1]) / 100

count =0
for i in seq_aligned:
    for j in i:
        if j=='-':
            count +=1
pa['-'] = count/(len(seq_aligned)*len(seq_aligned))
Nseq = len(seq_aligned)
alpa = Nseq - 1
beta = sqrt(Nseq)

# calculate the pua
pua = {}
for key in fua:
    t = []
    for i in range(len(seq_aligned[0])):
        t.append((alpa * fua[key][i] + beta * pa[key]) / (alpa + beta))

    pua[key] = t

# calculate the mua, the final PSSM matrix

mua = {}
for key in pua:
    t = []
    for i in range(len(seq_aligned[0])):
        t.append(log(pua[key][i] / pa[key]))
    mua[key] = t

# print_fua(mua)

# print(max_value())

## Calculate the gap penalty for every position
1. first, calculate the frequency of '-' in every position
2. Because for different position, the frequency is different. The bigger frequency should have lower penalty score.
3. I tested several functions and finally choose **$(tanh(frequency)-0.8)$**, and it gave me the promissing results.

In [5]:
gap1=[]
for i in range(len(seq_aligned[0])):
    gap1.append(fre('-',i))
# print(gap1)
gap =[(tanh(i)-0.8) for i in gap1]
print(gap)
# print(pua['A'])
# print_fua(pua)

[-0.77975984958469369, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.78785484822070539, -0.79190301096458215, -0.79595143912404431, -0.30511431692859925, -0.49797997873463395, -0.80000000000000004, -0.74741696521396317, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.80000000000000004, -0.79595143912404431, -0.76357887005982339, -0.74741696521396317, -0.063256172812858003, -0.063256172812858003, -0.063256172812858003, -0.063256172812858003, -0.047039270257138388, -0.047039270257138388, -0.047039270257138388, -0.047039270257138388, -0.047039270257138388, -0.047039270257138388, -0.040111394559352309, -0.040111394559352309, -0.040111394559352309, -0.040111394559352309, -0.040111394559352309, -0.047039270257138388, -0.047039270257138388, -0.047039270257138388, -0.047039270257138388, -0.047039270257138388, -0.04703927

To make it more clear we print out the largest value in the specific positions

In [6]:
# print the max value in the matrix and compare that to weblogo
def max_value(dict=pua):
    result = []
    for i in range(len(seq_aligned[0])):
        t = [dict[key][i] for key in dict]
        result.append([i,amino[argmax(t)],max(t)])
    return result
for i in max_value():
    print(i[0],":",i[1],'{0:2f}'.format(i[2]),end=" ")

0 : P 0.295855 1 : L 0.690778 2 : P 0.684012 3 : P 0.333910 4 : G 0.567460 5 : W 0.925382 6 : E 0.582478 7 : E 0.304679 8 : R 0.265898 9 : I 0.132947 10 : D 0.440907 11 : P 0.227357 12 : S 0.125738 13 : D 0.102221 14 : G 0.776761 15 : R 0.429533 16 : V 0.266696 17 : Y 0.698153 18 : Y 0.625849 19 : V 0.285724 20 : N 0.512370 21 : H 0.526517 22 : N 0.303070 23 : T 0.478896 24 : R 0.029959 25 : K 0.030133 26 : V 0.030758 27 : T 0.029851 28 : G 0.023279 29 : E 0.019269 30 : N 0.017660 31 : D 0.018501 32 : T 0.018435 33 : R 0.018543 34 : L 0.009600 35 : G 0.008057 36 : S 0.007769 37 : L 0.009600 38 : Q 0.006165 39 : E 0.019269 40 : S 0.019185 41 : V 0.019341 42 : P 0.021862 43 : S 0.019185 44 : Y 0.016975 45 : N 0.006244 46 : H 0.008974 47 : I 0.034005 48 : N 0.055715 49 : R 0.273509 50 : T 0.338093 51 : T 0.627309 52 : T 0.357121 53 : W 0.761747 54 : E 0.388399 55 : D 0.300105 56 : P 0.919951 57 : R 0.311563 58 : L 0.218901 

![](https://ww2.sinaimg.cn/large/006tNbRwgy1fdo5zwxfkdj30hn0603yt.jpg)
compare the max value in every position with the weblogo, it is very similar

The logo obtained from [PFAM](http://pfam.xfam.org/family/PF00397#tabview=tab4)
![](https://ww1.sinaimg.cn/large/006tNbRwgy1fdopkfdncvj31kw0gzk08.jpg)

difference: in the logo from PFAM family, the positions where  gaps have largest probabilities are not shown

## Initialize the score_mat and direc_mat

In [7]:
seq_list = read_fasta("protein-sequences.fasta")
seq1 = seq_list[0]
# initialize the score_mat and direc_mat
lseq2 = len(mua['A'])
score_mat = []
direc_mat = []

for i in range(len(seq1)+1):
    direc_mat.append([[0, 0, 0] for i in range(lseq2)])
for i in range(len(seq1)+1):
    score_mat.append([0 for i in range(lseq2)])


def print_mat2(alist):
    for i in alist:
        for j in i:
            print('{0:4f}'.format(j), end=" ")
        print()


def print_mat3(alist, n=3):
    for i in alist:
        for j in i:
            print(j[0:n], end='')
        print()

## Calculate the score according to the formula

In [8]:
def local_pssm(istart=1, jstart=1):
    for i in range(istart, len(seq1)):
        for j in range(jstart,lseq2):
            t1 = score_mat[i - 1][j - 1] + mua[seq1[i]][j]

            t2 = score_mat[i][j - 1] + gap[j-1]

            ## ??? doubt
            t3 = score_mat[i - 1][j] + gap[j]

            max_score = max(t1, t2, t3, 0)
            score_mat[i][j] = max_score

            if max_score > 0:
                if max_score == t1:
                    direc_mat[i][j][2] = 1
                if max_score == t2:
                    direc_mat[i][j][1] = 1
                if max_score == t3:
                    direc_mat[i][j][0] = 1

In [9]:
def print_pathpair(alist):
    for path in alist:
        path_u = ''.join(path[0])
        path_d = ''.join(path[1])

        print(path_u)
        print(path_d)

## Traceback
- very similar to local_traceback
- find the largest element in the last column(different from the local alignment)
- do traceback like that in local alignment

In [10]:
def traceback(k=1):
    path_pair = []
    path_down = []
    path_up=[]

    # find the largest element in the j column
    t = [score_mat[i][lseq2-1] for i in range(len(seq1))]
    i = argmax(t)

    recal_pair=[]


    print('score:',score_mat[i][lseq2-1])
    print("the coordinate of the starting tracing coordinate:", i)

    queue = []

    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, lseq2-1])
    while (len(queue) > 0):

        t = queue.pop(0)
        i = t[2]
        j = t[3]
        path_up = t[0]
        path_down = t[1]


        if score_mat[i][j]==0:
            path_pair.append([path_up,path_down])
        # scan all possible path and append it to the queue
        else:
            if direc_mat[i][j][1] == 1:
                path_up.insert(0, '-')
                j = j - 1

                path_down.insert(0, '*')

                recal_pair.append([i,j])

                if len(queue) < k:
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                j = j + 1
                path_up = path_up[1:]
                path_down = path_down[1:]
            if direc_mat[i][j][0] == 1:
                path_down.insert(0, '-')
                i = i - 1
                path_up.insert(0, seq1[i])


                recal_pair.append([i,j])

                if len(queue) < k:
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                i = i + 1
                path_up = path_up[1:]
                path_down = path_down[1:]

            if direc_mat[i][j][2] == 1:
                i = i - 1
                j = j - 1
                path_up.insert(0, seq1[i])
                path_down.insert(0, '*')

                recal_pair.append([i,j])

                if len(queue) < k:
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                i = i + 1
                j = j + 1
                path_up = path_up[1:]
                path_down = path_down[1:]

            score_mat[i][j]=0

    #print(path_pair)

    #print("recal_pair:",recal_pair)

    for k in range(len(recal_pair) - 1, -1, -1):
        t = recal_pair[k]
        recal(recal_pair[k][0], recal_pair[k][1])

    return path_pair

In [11]:
def recal(i,j):
    score_mat[i][j] = 0
    if i< len(seq1)-2 and j <lseq2-2:
        if direc_mat[i+1][j][0]==0 and direc_mat[i][j+1][1]==0 and direc_mat[i+1][j+1][2]==0:
            return
    if i< len(seq1)-2:
        i = i+1
        if direc_mat[i][j][0]==1:
            # do recalculate
            t1 = score_mat[i - 1][j - 1] + mua[seq1[i]][j]

            t2 = score_mat[i][j - 1] + gap[j - 1]

            
            t3 = score_mat[i - 1][j] + gap[j]
            max_score = max(t1, t2, t3, 0)
            score_mat[i][j] = max_score

            if max_score > 0:
                if max_score == t1:
                    direc_mat[i][j][2] = 1


                elif max_score == t2:
                    # horizental
                    direc_mat[i][j][1] = 1

                else:
                    # vertical, seq1 has a gap
                    direc_mat[i][j][0] = 1
            else:
                direc_mat[i][j][0]=0
                direc_mat[i][j][1]=0
                direc_mat[i][j][2]=0
            recal(i,j)


        i = i-1

    if j < lseq2-2:
        j = j+1
        if direc_mat[i][j][1] ==1:
            t1 = score_mat[i - 1][j - 1] + mua[seq1[i]][j]

            t2 = score_mat[i][j - 1] + gap[j - 1]

            ## ??? doubt
            t3 = score_mat[i - 1][j] + gap[j]

            max_score = max(t1, t2, t3, 0)
            score_mat[i][j] = max_score

            if max_score > 0:
                if max_score == t1:
                    direc_mat[i][j][2] = 1


                elif max_score == t2:
                    # horizental
                    direc_mat[i][j][1] = 1

                else:
                    # vertical, seq1 has a gap
                    direc_mat[i][j][0] = 1
            else:
                direc_mat[i][j][0]=0
                direc_mat[i][j][1]=0
                direc_mat[i][j][2]=0
            recal(i, j)
        j = j-1

    if i<len(seq1)-2 and j<lseq2-1:
        i = i+1
        j = j+1
        if direc_mat[i][j][2] ==1:
            t1 = score_mat[i - 1][j - 1] + mua[seq1[i]][j]

            t2 = score_mat[i][j - 1] + gap[j - 1]

            ## ??? doubt
            t3 = score_mat[i - 1][j] + gap[j]

            max_score = max(t1, t2, t3, 0)
            score_mat[i][j] = max_score

            if max_score > 0:
                if max_score == t1:
                    direc_mat[i][j][2] = 1

                elif max_score == t2:
                    # horizental
                    direc_mat[i][j][1] = 1
                else:
                    # vertical, seq1 has a gap
                    direc_mat[i][j][0] = 1
            else:
                direc_mat[i][j][0]=0
                direc_mat[i][j][1]=0
                direc_mat[i][j][2]=0
            recal(i, j)


        i = i -1
        j = j -1

In [12]:
# pack local_pssm which calculate the score, traceback(k) and print_pathpair in one function
# the n indicates the number of motifs we would like to find, k indicates in one path, the maximum number of subpaths
def multi_path_pssm(n,k):
    local_pssm()
    for i in range(n):
        print_pathpair(traceback(k))
        
multi_path_pssm(3,2)

score: 48.5418325556
the coordinate of the starting tracing coordinate: 262
PLPDGWEQAMTQ-DGEIYYINHK--------------------------NKTTSWLDPR
*********************-*************************************
score: 47.6561536004
the coordinate of the starting tracing coordinate: 203
PLPAGWEMAKTS-SGQRYFLNHI-------------------------DQTTTWQDPR
**********************************************************
score: 47.4035814071
the coordinate of the starting tracing coordinate: 202
-
*


The result from [Uniprot-P46937](http://www.uniprot.org/uniprot/P46937)
![](https://ww2.sinaimg.cn/large/006tNbRwgy1fdoq478hhsj31kw07twgl.jpg)

The result given from the website is exactly the same with the result from my algorithms